# 知识工程-作业2-第二部分

2024214500 叶璨铭


## 补充完成`data_util.py`中数据加载部分

首先 main.py 中
from data_utils import sent2features, sent2labels, sent2tokens, read_examples_from_file

调用了 data_utils的sent2features

然后 data_utils 中sent2features其实是通过 word2features 实现

助教的 sent 是 sentence的缩写，不是发射的缩写。

需要特别注意预处理代码中是用 两个换行 约定来分隔句子，一个换行分隔单词。


现在可以实现 read_examples_from_file 了

In [ ]:
#| export
from typing import List
from uuid import uuid4

def read_examples_from_file(
    file_path: str,  # file path to load
    mode: str,  # "train" or "test"
) -> List[InputExample]:
    """
    Read file and load into a list of `InputExample`s
    """
    examples = []

    with open(file_path, "r", encoding="utf-8") as f:
        file = f.read()

    for sentence in file.strip().split("\n\n"):
        words = []
        labels = [] if mode!="real_test" else None
        for line in sentence.split("\n"):
            word, label = line.split()
            words.append(word)
            if mode!="real_test":
                labels.append(label)
        examples.append(InputExample(guid=str(uuid4()), words=words, labels=labels))

    return examples

虽然助教的注释说test模式不要返回label，但是实际上我们的main中需要label去评测，所以我增加了一个“real_test"满足需求

### 特征的构造

首先回顾老师的课件
![](image-4.png)

课件提到了最大熵模型构造特征，
比如x在“in”的后面，那就意味着x和LOCATION具有一定的关系。

最大熵模型的特征是有x和y的，因为是对每一个可能的y去做匹配，然后看看和哪个y的关系大。我们这里是crf，特征就是对x提取。

可以看出，对x的提取需要一定的语言学知识，课件上并没有详细介绍，我们需要自己去找网络资料，看看哪些提取方式比较合理。

助教代码注释提示我们可以参考 crfsuite的文档，于是我们找到了官方实例 https://github.com/scrapinghub/python-crfsuite/blob/master/examples/CoNLL%202002.ipynb

这里
```python
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag,
        'postag[:2]=' + postag[:2],
    ]
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:postag=' + postag1,
            '-1:postag[:2]=' + postag1[:2],
        ])
    else:
        features.append('BOS')
        
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:postag=' + postag1,
            '+1:postag[:2]=' + postag1[:2],
        ])
    else:
        features.append('EOS')
                
    return features
```

然而我们这里助教让我们处理的格式实际上是单个字，首先word[-3:]就没有意义了，这个在英语中原本是想要提取 "est" "ly"“之类的东西。

而其他的 istitle isupper 也是对中文来说用处不大。助教给的数据集和CoNLL不一样，默认并没有 postag 。

于是我决定搜索一下处理中文特征的一些NER论文，AI搜索给我了 https://pdfs.semanticscholar.org/b760/e6abd21e1f5db388e911bb4d8a8581c634b7.pdf?skipShowableCheck=true https://pdfs.semanticscholar.org/a063/601d35146fd5f4fa92eeaf3aac01243fc852.pdf?skipShowableCheck=true

对于CRF而言，本来就是考虑这个线性条件随机场，关注的是左右两个单词，所以不管中文英文，首先有
- 当前字符（word[i]）
- 前一个字符（word[i-1]）
- 后一个字符（word[i+1]）
- 前一个和当前字符的组合（word[i-1]+word[i]）
- 当前和后一个字符的组合（word[i]+word[i+1]）

这是一类特征，然后是边界条件，这个上面的代码中也有了，BOS和EOS。


In [11]:
from data_utils import InputExample

In [ ]:
#| export
def word2features(sent: InputExample, # InputExample
                i: int, # index for target word
                use_simple_feature_only: bool = True # 
                )-> List[str]:
    """
    get discrete features for a single word
    Please design features for one word in the sentence as input into pycrfsuite model (https://python-crfsuite.readthedocs.io/en/latest/)
    """
    word = sent.words[i]
    # if not hasattr(sent, "postags"):
        
    features = single_word2features(word)
    if i > 0:
        word1 = sent.words[i-1]
        features += single_word2features(word1, "-1:")
    else:
        features.append('BOS') # 句子开头
        
    if i < len(sent)-1:
        word1 = sent.words[i+1]
        features += single_word2features(word1, "+1:")
    else:
        features.append('EOS')
                
    return features


对于中文来说比较厉害的是什么呢？其实是
- 拼音
- 偏旁部首
- 笔画数
- 是否为百家姓，比如{'王','李','张','刘','陈'},
- 是否带有行政区划，比如{'省','市','区','县','镇'},
- 是否是组织，比如{'公司','局','银行','医院'}

具体来说上面这些特征需要有库支持我才能提取，我们一个一个来看

首先拼音很好实现，https://github.com/mozillazg/python-pinyin



In [10]:
from pypinyin import pinyin
pinyin("你好")

[['nǐ'], ['hǎo']]

接下来是 git+https://github.com/wangchuan2008888/cn-radical.git

我们最后安装了
```bash
uv pip install pypinyin git+https://github.com/wangchuan2008888/cn-radical.git
```

## 补充完成`main.py`中模型定义部分

我们这里直接参考官方文档的实例代码 https://github.com/scrapinghub/python-crfsuite/blob/master/examples/CoNLL%202002.ipynb

把verbose换为True

```python
# training
    #############
    # buld trainer with pycrfsuite and set related hyper-parameters
    import pycrfsuite

    trainer = pycrfsuite.Trainer(verbose=True)
    trainer.set_params(
        {
            "c1": 1.0,  # coefficient for L1 penalty
            "c2": 1e-3,  # coefficient for L2 penalty
            "max_iterations": 50,  # stop earlier
            # include transitions that are possible, but not observed
            "feature.possible_transitions": True,
        }
    )
    ##################

```

## 不同参数运行模型，对比性能

我们加入argparse，方便对比
```python
argparser = argparse.ArgumentParser()
argparser.add_argument("--mode", default="bio", type=str)
argparser.add_argument("--use_simple_feature_only", action="store_true")
args = argparser.parse_args()

model = f"./msra_{mode}_{simple}.crfsuite"
```

现在我们可以运行

In [7]:
!python -u main.py --mode bio --use_simple_feature_only

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 35631
Seconds required: 2.007

L-BFGS optimization
c1: 1.000000
c2: 0.001000
num_memories: 6
max_iterations: 50
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 1179816.490259
Feature norm: 1.000000
Error norm: 269111.903067
Active features: 32447
Line search trials: 1
Line search step: 0.000000
Seconds required for this iteration: 2.054

***** Iteration #2 *****
Loss: 1168183.090147
Feature norm: 1.002591
Error norm: 259493.229277
Active features: 29538
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 1.030

***** Iteration #3 *****
Loss: 985128.689679
Feature norm: 1.596923
Error norm: 443345.153573
Active features: 22519
Line search trials: 1
Line search step: 1.000000
Seconds requi

第一次运行的时候报错
```python
Traceback (most recent call last):
  File "/home/ye_canming/repos/assignments/THU-Coursework-Knowledge-Engineering/2.命名实体识别的作业/main.py", line 67, in <module>
    results, results_by_tag = evaluator.evaluate()
ValueError: too many values to unpack (expected 2)
```

这其实是因为nervaluate 版本更新了，根据最新的文档
https://pypi.org/project/nervaluate/
应该这样写

`results, results_per_tag, result_indices, result_indices_by_tag = evaluator.evaluate()`



### 运行结果分析
解决了问题之后，我们可以看到bio+基础特征的结果f1达到 0.78


In [ ]:
# !python -u main.py --mode bio --use_simple_feature_only
{
    "ent_type": {
        "correct": 4507,
        "incorrect": 502,
        "partial": 0,
        "missed": 1172,
        "spurious": 314,
        "possible": 6181,
        "actual": 5323,
        "precision": 0.8467029870373849,
        "recall": 0.7291700372108073,
        "f1": 0.7835535465924895,
    },
}


In [ ]:
!python -u main.py --mode bioes --use_simple_feature_only

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 42083
Seconds required: 2.116

L-BFGS optimization
c1: 1.000000
c2: 0.001000
num_memories: 6
max_iterations: 50
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 1445234.599760
Feature norm: 1.000000
Error norm: 1000714.164890
Active features: 37932
Line search trials: 1
Line search step: 0.000000
Seconds required for this iteration: 3.590

***** Iteration #2 *****
Loss: 1335224.256168
Feature norm: 1.182200
Error norm: 285825.629725
Active features: 34789
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 1.769

***** Iteration #3 *****
Loss: 1315229.737326
Feature norm: 1.227057
Error norm: 251970.033112
Active features: 34801
Line search trials: 1
Line search step: 1.000000
Seconds req

In [ ]:
# 
{
    "ent_type": {
        "correct": 3919,
        "incorrect": 881,
        "partial": 0,
        "missed": 1232,
        "spurious": 403,
        "possible": 6032,
        "actual": 5203,
        "precision": 0.7532192965596771,
        "recall": 0.6497015915119363,
        "f1": 0.6976412995104584,
    },
}


可以看到简单的特征效果并没有达到助教说的84。现在我们把高级特征加上。